# Code to measure the offset of each MIRI galaxy to the centre of the frame

To do this, we will read in the catalogue of galaxies to obtain their respective IDs, RAs and Decs. Then we will fit a centroid to all our MIRI images and determine their coordinates. From this we can calculate the spherical offsets. Lastly we will plot the results and see what happens.

In [ ]:
from astropy.io import fits
from astropy import table
import numpy as np
from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import astropy.units as u
from astropy.nddata import Cutout2D
from photutils import centroids
import scipy
import os
import glob
import pandas as pd

Load the necessary data files

In [ ]:
# Load target catalogue
with fits.open('./cat_targets.fits') as cat_hdul:
    #cat_hdul.info()
    cat_data = cat_hdul[1].data  # Extract data from table
    cat_header = cat_hdul[1].header
    ids = cat_data['id']
    ra = cat_data['ra']
    dec = cat_data['dec']

cat = table.Table.read('./cat_targets.fits', 1)

df = pd.DataFrame(cat_data)

# Adjust pandas display settings
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.max_rows", None)  # Show all rows
pd.set_option("display.max_colwidth", None)  # Prevent text truncation
print(df)

# add columns to store the astrometric offset for PRIMER and COSMOS-Web
cat['primer_dra'] = 0.0*u.arcsec
cat['primer_ddec'] = 0.0*u.arcsec
cat['cweb_dra'] = 0.0*u.arcsec
cat['cweb_ddec'] = 0.0*u.arcsec
print(cat.columns)

# Specify the path to the cutouts directory
my_cutouts = '/home/bpc/University/master/Red_Cardinal/cutouts/'

# Load the PRIMER cutouts
primer_cutouts = glob.glob(os.path.join(my_cutouts, "*primer.fits"))
print(f"Found {len(primer_cutouts)} FITS files for the PRIMER survey.")

# Load the COSMOS-Web cutouts
cweb_cutouts = glob.glob(os.path.join(my_cutouts, "*cweb.fits"))
print(f"Found {len(cweb_cutouts)} FITS files for the COSMOS-Web survey.")

cat


# Loop through the cutouts and calculate the offset
For each of the two surveys we will loop through the cutouts and calculate the offset with respect to the catalogue.

In [ ]:

for i,g in enumerate(cat):

    cutout_side = 2.5*u.arcsec
    good_frac_cutout = 0.7
    smooth_sigma = 1.0
    
    """
    if g['source_id'] == 16424: # faint source, use bright source nearby
        cutout_side = 2.8*u.arcsec
        good_frac_cutout = 0.9
    if g['source_id'] == 7904: # exclude bright source nearby
        good_frac_cutout = 0.4
    if g['source_id'] == 11716: # exclude bright source nearby
        good_frac_cutout = 0.4
    if g['source_id'] == 21472: # fuzzy source, smooth more
        smooth_sigma=2.0
    """    
    
    print(g['id'])
    ref_position = SkyCoord(ra=g['ra'], dec=g['dec'], unit=u.deg)
    cutout_size = (cutout_side, cutout_side)

    # HANDLE miri DATA -----------------------------------------------------------------

    # extract cutout from the PRIMER cutouts I already made
    primer_F770W_cutout_name =  f"./cutouts/{g['id']}_F770W_cutout_primer.fits"
    #primer_F1800W_cutout_name = f"./cutouts/{g['source_id']}_F1800W_cutout_primer.fits"
    
    try:
        with fits.open(primer_F770W_cutout_name) as hdu_F770W:
            #hdu_F770W.info()
            F770W_data = hdu_F770W[1].data
            F770W_header = hdu_F770W[1].header
            F770W_wcs = WCS(F770W_header)
        #hdu_primer_F1800W = fits.open(primer_F770W_cutout_name)[0]
    except FileNotFoundError:
        print(f"No PRIMER cutout found for source {g['id']}. Skipping.")
        continue

    cutout_F770W = Cutout2D(F770W_data, ref_position, cutout_size, wcs=F770W_wcs)
    #cutout_hst.wcs.wcs.radesys = 'ICRS' # change to ICRS, which is the frame of the reference mosaic
    
    
    # HANDLE NIRCAM DATA ---------------------------------------------------------------
    
    # extract the cutouts from the NIRCam photometry
    nircam_frame = f'./NIRCam/F444W_cutouts/{g['id']}_F444W_cutout.fits'
    
    # Read in cutout for that specific galaxy from NIRCam
    with fits.open(nircam_frame) as hdu_nircam:
        #hdu_nircam.info()
        nircam_data = hdu_nircam[1].data
        nircam_header = hdu_nircam[1].header
        nircam_wcs = WCS(nircam_header)
        
    cutout_nircam = Cutout2D(nircam_data, ref_position, cutout_size, wcs=nircam_wcs)
    
    # CALCULATE THE OFFSETS OF THE CENTROIDS -------------------------------------------    
    
    # find centroid in NIRCam image
    fig, axs = plt.subplots(1,2, figsize=[10,5])
    cutout_nircam_smooth = scipy.ndimage.gaussian_filter(cutout_nircam.data, smooth_sigma)
    axs[0].imshow(cutout_nircam_smooth)
    ref_pix = cutout_nircam.wcs.world_to_pixel(ref_position)
    centroid_nircam_pix = centroids.centroid_quadratic(
        cutout_nircam_smooth, 
        xpeak=cutout_nircam.data.shape[0]//2, ypeak=cutout_nircam.data.shape[1]//2, 
        search_boxsize=int(np.floor(good_frac_cutout*cutout_nircam_smooth.shape[0])//2*2+1), fit_boxsize=5)
    centroid_nircam = cutout_nircam.wcs.pixel_to_world(centroid_nircam_pix[0], centroid_nircam_pix[1])
    axs[0].plot(centroid_nircam_pix[0], centroid_nircam_pix[1], 'x', color='red')
    nircam_title = f"NIRCam F440W reference image {g['id']}"
    axs[0].set(title=nircam_title)
    
    # find centroid in PRIMER cutouts
    cutout_F770W_smooth = scipy.ndimage.gaussian_filter(cutout_F770W.data, smooth_sigma)
    axs[1].imshow(cutout_F770W_smooth)
    ref_pix = cutout_F770W.wcs.world_to_pixel(ref_position)
    centroid_F770W_pix = centroids.centroid_quadratic(
        cutout_F770W_smooth, 
        xpeak=cutout_F770W.data.shape[0]//2, ypeak=cutout_F770W.data.shape[1]//2, 
        search_boxsize=int(np.floor(good_frac_cutout*cutout_F770W_smooth.shape[0])//2*2+1), fit_boxsize=5)
    centroid_F770W = cutout_F770W.wcs.pixel_to_world(centroid_F770W_pix[0], centroid_F770W_pix[1])
    axs[1].plot(centroid_F770W_pix[0], centroid_F770W_pix[1], 'o', color='orange')
    miri_title = f"MIRI F770W cutout {g['id']}"
    axs[1].set(title=miri_title)

    # show expected position of the centroid
    expected_position_pix = cutout_F770W.wcs.world_to_pixel(centroid_nircam)
    axs[1].plot(expected_position_pix[0], expected_position_pix[1], 'x', color='red')

    # if the fit does not find a centroid
    if np.isnan(centroid_nircam_pix).any() or np.isnan(centroid_F770W_pix).any():
        print("CENTROID NOT FOUND!")
        break
    
    # Create output directory
    output_dir = "./offsets/PRIMER_F770W/"
    os.makedirs(output_dir, exist_ok=True)

    figname = output_dir + f"{g['id']}_F770W_alignment_primer.pdf"
    fig.savefig(figname)
    plt.close()

    print(f'Successfully saved figure {figname} to {output_dir}!')
    
    # calculate offset and store in the catalog
    dra, ddec = centroid_nircam.spherical_offsets_to(centroid_F770W)
    print(dra.to(u.arcsec), ddec.to(u.arcsec))
    cat['primer_dra'][i] = dra.to(u.arcsec).value
    cat['primer_ddec'][i] = ddec.to(u.arcsec).value


In [ ]:
cat

In [ ]:

# Open your NIRCam FITS file
with fits.open(nircam_frame) as hdu_nircam:
    #hdu_nircam.info()
    nircam_data = hdu_nircam[1].data
    nircam_header = hdu_nircam[1].header
    nircam_wcs = WCS(nircam_header)
    print("NIRCam Reference Frame:", nircam_header.get('RADESYS', 'Not Found'))
    print("NIRCam Equinox:", nircam_header.get('EQUINOX', 'Not Found'))

# Open your MIRI FITS file
primer_F770W_cutout_name =  f"./cutouts/16419_F770W_cutout_primer.fits"
with fits.open(primer_F770W_cutout_name) as hdu_F770W:
    #hdu_F770W.info()
    miri_data = hdu_F770W[1].data
    miri_header = hdu_F770W[1].header
    miri_wcs = WCS(F770W_header)
    print("MIRI Reference Frame:", miri_header.get('RADESYS', 'Not Found'))
    print("MIRI Equinox:", miri_header.get('EQUINOX', 'Not Found'))
